In [1]:
# Esto agrega al python path el directorio ..
#! coding: utf-8
%pylab inline
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config
from dataframe_builder import DataFrameBuilder

builder = DataFrameBuilder()
df = builder.build()

Populating the interactive namespace from numpy and matplotlib
Buscando dataframe en /home/marat/projects/tp-aa/cache/dataframe.pickle
Encontrado. Dimensiones: (80995, 56)



Conjunto a probar

1)C in range [0:20] Dice el manual:Por default es uno. If you have a lot of noisy observations you should decrease it. It corresponds to regularize more the estimation.

2)Kernels {'rbf ' ,' linear' ,' poly' ,'sigmoid'}  

3) para poly podemos hacer un range de degrees, esa opcion va a ser ignorada por los otros kernels asi que podemos hacer degree in range[2:5]. Podemos elegis=r ademas si agregamos un coef0, o sea el termino independiente, no veo por que seria importnte por ahora eso.

4)(REVISAR)Por default hay un parametro epsilon=0.1. Epsilon indica un rango de distancia que no sera considerado como penalidad, si el vector esta a espilon--ver

decision_function(X)[source]
Distance of the samples X to the separating hyperplane.
Parameters:	
X : array-like, shape (n_samples, n_features)
returns:	
X : array-like, shape (n_samples, n_classes * (n_classes-1) / 2)
Returns the decision function of the sample for each class in the model. If decision_function_shape=’ovr’, the shape is (n_samples, n_classes)

-Cuando fiteamos este modelo podemos elegir un sample weight, pero no creo que lo necesitemos ya que estan balanceados.



In [2]:
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.metrics.pairwise import laplacian_kernel


options={
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid'], # poly tarda mucho, por ahi seria mejor hacerlo aparte
    'C' : np.arange(0.1, 3, 0.1),
    'degree' : range(2, 9),
    #'gamma' : np.arange(0,1,0.1)
}


con los atributos.

En el manual de sklearn dice: Proper choice of C and gamma is critical to the SVM’s performance. One is advised to use sklearn.grid_search.GridSearchCV with C and gamma spaced exponentially far apart to choose good values.


Attributes:	
n_support_ : array-like, dtype=int32, shape = [n_class]
Number of support vectors for each class.


In [ ]:
from search import find_best_classifier

classifier = find_best_classifier(
    df, SVC, clf_args={'max_iter':150}, param_distributions=options, 
    scoring='roc_auc', cv=10, n_jobs=6, n_iter=300)    

Buscando parámetros para SVC


Process PoolWorker-14:
Process PoolWorker-13:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/marat/.pyenv/versions/2.7.11/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/marat/.pyenv/versions/2.7.11/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
  File "/home/marat/.pyenv/versions/2.7.11/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/marat/.pyenv/versions/2.7.11/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
  File "/home/marat/.pyenv/versions/2.7.11/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    task = get()
  File "/home/marat/.pyenv/versions/2.7.11/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/home/marat/.pyenv/versions/2.7.11/envs/aprendizaje/lib/python2.7/site-packages/sklearn/externals/joblib/

In [ ]:
get_scores(classifier)